# Database for Scientific Knowledge 

>  A local Postgresql database of scientific content with associated information generated by the agent. The intent is to use this repository as 'memory' in a Langchain-enabled agent.

In [ ]:
#| default_exp utils.ceifns_db

In [ ]:
#| hide
from nbdev import *

We define a local database for use with the Alhazen schema conforming to the following schema:

![Schema](../pics/schema/Alhazen_Schema_v0.0.3.png) 

The key elements of the schema are:

**Abstract Parent Class**

* *InformationContentEntity* - a named piece of information (all other entities inherit from this)
  
**Classes Denoting Extant Scientific Knowledge from External Sources**

* *ScientificKnowledgeCollection* - a collection of scientific knowledge expressions
* *ScientificKnowledgeExpression* - an expression of scientific knowledge (e.g., a paper, a database record, a blog post, etc.) 
* *ScientificKnowledgeItem* - the data item that manifests the expression (e.g., a pdf, an html data file, a yaml file, etc.) 
* *ScientificKnowledgeFragment* - a relevant fragment of the item (e.g., a paragraph, a table, a figure, etc.)            

**Classes Denoting Information Generated by Alhazen**

* *Note* - an annotation placed on any InformationContentEntity  (e.g., a comment, a question, a link to another fragment, etc.)

> Our definitions are inspired by [FABIO](http://www.sparontologies.net/ontologies/fabio), but only differentiate between '*expression*' (as the reference to an scientific underlying work product, e.g., a paper or database record) and '*item*' (as the actual data that delivers on that expression). 
> The system is coded in LinkML and generated as a Postgresql database and a SQLAlchemy ORM.

In [ ]:
#| export

from alhazen.aliases import *
from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import ESearchQuery, EuroPMCQuery, load_paper_from_openalex, read_references_from_openalex 
from alhazen.utils.pdf_research_article_text_extractor import LAPDFBlockLoader, HuridocsPDFLoader
from alhazen.utils.html_research_article_text_extractor import TrafilaturaSectionLoader

#from alhazen.utils.queryTranslator import QueryTranslator, QueryType
from alhazen.schema_sqla import *
import alhazen.schema_python as linkml_py
from alhazen.utils.jats_text_extractor import NxmlDoc


from bs4 import BeautifulSoup

from dataclasses import dataclass, field
from datetime import datetime
import fitz
from importlib_resources import files
import json

from langchain.pydantic_v1 import BaseModel, Extra, Field, root_validator
from langchain.docstore.document import Document
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings
from langchain.schema.embeddings import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector

import local_resources.linkml as linkml
import local_resources.data_files.world_bank_wdi as wdi

import nltk.data
import os

import pandas as pd
from pathlib import Path
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import random
import re
import requests
from sqlalchemy import create_engine, exists, Engine, text, func, or_, desc
from sqlalchemy.orm import sessionmaker, aliased, Session
import sqlite3  
import subprocess
import sys
from time import time,sleep
import torch
from tqdm import tqdm
import yaml
import uuid
from langchain.vectorstores.pgvector import PGVector

from zipfile import ZipFile


In [ ]:
#| export

def create_ceifns_database(db_name):
  """Use this function to create a CEIFNS database within the local postgres server."""
  conn = psycopg2.connect(
    database="postgres",
      user=os.environ['POSTGRES_USER'],
      password=os.environ['POSTGRES_PASSWORD'],
      host=os.environ['POSTGRES_HOST'],
      port= '5432'
  )
  conn.autocommit = True
  cursor = conn.cursor()
  cursor.execute('CREATE database ' + db_name)
  conn.close()
  
  conn = psycopg2.connect(
    database=db_name,
    user=os.environ['POSTGRES_USER'],
    password=os.environ['POSTGRES_PASSWORD'],
    host=os.environ['POSTGRES_HOST'],
    port= '5432'
  )
  conn.autocommit = True
  cursor = conn.cursor()
  with open(files(linkml).joinpath('schema.sql'), 'r') as f:
    for sql in tqdm(f.read().split(';')):
        if len(sql.strip()) == 0:
          continue
        cursor.execute(sql)

  engine = create_engine("postgresql+psycopg2://%s:%s@%s:5432/%s"%(os.environ['POSTGRES_USER'], os.environ['POSTGRES_PASSWORD'], os.environ['POSTGRES_HOST'], db_name))
  session_class = sessionmaker(bind=engine)
  session = session_class()

  # add tabular data about countries
  url = 'https://www.iban.com/country-codes'
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  table = soup.find('table')
  rows = table.find_all('tr')
  countries = []
  for row in rows:
    cols = row.find_all('td')
    if len(cols)==0:
        continue
    country = cols[0].text.strip()
    code3 = cols[1].text.strip()
    code2 = cols[2].text.strip()
    countries.append((country, code3, code2))
  country_df = pd.DataFrame(countries, columns=['country', 'code2', 'code3'])
  wdi_classes_path = files(wdi).joinpath('CLASS.xlsx')
  wdi_xls = pd.ExcelFile(wdi_classes_path)
  wdi_df = wdi_xls.parse('List of economies')
  for i, crow in country_df.iterrows():
    
    crow2 = wdi_df[wdi_df['Code']==crow.code3]
    if len(crow2)>0:
      country_region = str(crow2['Region'].values[0])
      country_income = str(crow2['Income group'].values[0])
      c = Country(id=uuid.uuid4().hex[0:16],
                  name = crow.country, 
                  type='Country', 
                  code2 = crow.code2,
                  code3 = crow.code3,
                  region = country_region,
                  income = country_income)
    else: 
      c = Country(id=uuid.uuid4().hex[0:16],
            name = crow.country, 
            type='Country', 
            code2 = crow.code2,
            code3 = crow.code3)
    session.add(c)
  session.commit()
      
  conn.close()

def drop_ceifns_database(db_name, backupFirst=True):
  """Use this function to delete a CEIFNS database from the local postgres server. Set the backupFirst flag to True to backup the database before deletion."""
  if backupFirst:
    loc = os.environ['LOCAL_FILE_PATH']
    current_date_time = datetime.now()
    formatted_date_time = f'{current_date_time:%Y-%m-%d-%H-%M-%S}'
    backup_path = loc+db_name+'/backup'+formatted_date_time+'.sql'
    backup_ceifns_database(db_name, backup_path)
    print("Database has been backed up to %s"%(backup_path));
  conn = psycopg2.connect(
    database="postgres",
      user=os.environ['POSTGRES_USER'],
      password=os.environ['POSTGRES_PASSWORD'],
      host=os.environ['POSTGRES_HOST'],
      port= '5432'
  )
  conn.autocommit = True
  cursor = conn.cursor()
  cursor.execute("SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname='"+db_name+"' AND leader_pid IS NULL;")
  cursor.execute('DROP database ' + db_name)
  print("Database has been dropped successfully !!");
  conn.close()

# ADAPTED FROM https://gist.github.com/valferon/4d6ebfa8a7f3d4e84085183609d10f14
def backup_ceifns_database(db_name, dest_file, verbose=False):
  """
  Backup postgres db to a local file. Note that this
  """
  "postgresql://localhost:5432/"+db_name

  if verbose:
    try:
      process = subprocess.Popen(
          ['pg_dump',
            '--dbname=postgresql://localhost:5432/{}'.format(db_name),
            '-Fc',
            '-f', dest_file,
            '-v'],
          stdout=subprocess.PIPE
      )
      output = process.communicate()[0]
      if int(process.returncode) != 0:
        print('Command failed. Return code : {}'.format(process.returncode))
        exit(1)
      return output
    except Exception as e:
      print(e)
      exit(1)
  else:
    try:
      process = subprocess.Popen(
          ['pg_dump',
            '--dbname=postgresql://localhost:5432/{}'.format(db_name),
            '-f', dest_file],
          stdout=subprocess.PIPE
      )
      output = process.communicate()[0]
      if process.returncode != 0:
        print('Command failed. Return code : {}'.format(process.returncode))
        exit(1)
      return output
    except Exception as e:
      print(e)
      exit(1)

# ADAPTED FROM https://gist.github.com/valferon/4d6ebfa8a7f3d4e84085183609d10f14
def restore_ceifns_database(db_name, backup_file, verbose=False):
  """
  Restore postgres db from a file.
  """
  if verbose:
    try:
      process = subprocess.Popen(
          ['pg_restore',
            '--no-owner',
            '--dbname=postgresql://localhost:5432/{}'.format(db_name),
            '-v',
            backup_file],
          stdout=subprocess.PIPE
      )
      output = process.communicate()[0]
      if int(process.returncode) != 0:
        print('Command failed. Return code : {}'.format(process.returncode))

      return output
    except Exception as e:
      print("Issue with the db restore : {}".format(e))
  else:
    try:
      process = subprocess.Popen(
          ['pg_restore',
            '--no-owner',
            '--dbname=postgresql://localhost:5432/{}'.format(db_name),
            backup_file],
        stdout=subprocess.PIPE
      )
      output = process.communicate()[0]
      if int(process.returncode) != 0:
        print('Command failed. Return code : {}'.format(process.returncode))
      return output
    except Exception as e:
      print("Issue with the db restore : {}".format(e))

def list_databases():
  """
  List all CEIFNS databases in the postgres server
  """
  engine = create_engine("postgresql+psycopg2://%s:%s@%s:5432/%s"%(os.environ['POSTGRES_USER'], os.environ['POSTGRES_PASSWORD'], os.environ['POSTGRES_HOST'], 'postgres'))
  connection = engine.connect()
  result = connection.execute(text("SELECT datname FROM pg_database;"))
  dbn = [row[0] for row in result if row[0] != 'postgres']
  connection.close()

  ah_dbs = []
  for db in dbn:
    engine = create_engine("postgresql+psycopg2://%s:%s@%s:5432/%s"%(os.environ['POSTGRES_USER'], os.environ['POSTGRES_PASSWORD'], os.environ['POSTGRES_HOST'], db))
    try: 
        connection = engine.connect()
        result = connection.execute(text('SELECT * FROM "ScientificKnowledgeCollection" LIMIT 1;'))
    except Exception as e:
        continue
    connection.close()    
    ah_dbs.append(db)

  return ah_dbs

`<<<<<<< HEAD`

In [ ]:
#| export 

class Ceifns_LiteratureDb(BaseModel):
  """This class runs a set of queries on external literature databases to build a local database of linked corpora and papers.

  Functionality includes:

    * Executes queries over European PMC 
    * Can run combinatorial sets of queries using a dataframe structure
        * Requires a column of queries expressed in boolean logic
        * Optional to define a secondary spreadsheet with a column of subqueries expressed in boolean logic
    * Has capability to run boolean logic over sources (currently only European PMC, but possibly others) 
    * Builds a local Postgresql database with tables for collections, expressions, items, fragments, and notes. 
    * Provides an API for querying the database and returning results as sqlAlchemy objects.
    * Permits user to download a local copy of full text papers in NXML(JATS), PDF, and HTML format.
  """
  loc: str = Field()
  name: str = Field()
  engine: Engine = Field(default=None, init=False)
  session: Session = Field(default=None, init=False)
  sent_detector: nltk.tokenize.punkt.PunktSentenceTokenizer = Field(default=None, init=False)
  embed_model_name: str = Field(default='BAAI/bge-large-en-v1.5', init=False)
  embed_model_device: str = Field(default='cpu', init=False)
  embed_model: Embeddings = Field(default=None, init=False)

  class Config:
    """Configuration for this pydantic object."""
    arbitrary_types_allowed = True

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    
    if self.loc[-1] != '/':
      self.loc = self.loc + '/' 
    if os.path.exists(self.loc) is False:
      os.makedirs(self.loc)

    db_dir = Path(self.loc + self.name)
    if db_dir.exists() is False:
      os.makedirs(db_dir)

    self.engine = create_engine("postgresql+psycopg2://%s:%s@%s:5432/%s"%(os.environ['POSTGRES_USER'], os.environ['POSTGRES_PASSWORD'], os.environ['POSTGRES_HOST'], self.name))

    self.start_session() # instantiate session 

    log_path = '%s%s_log.txt'%(self.loc, self.name)
    if os.path.exists(log_path) is False:
      Path(log_path).touch()
    
    self.sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

    #
    # hard coded embedding model at present.
    #
    model_kwargs = {"device": self.embed_model_device}
    encode_kwargs = {"normalize_embeddings": True}
    self.embed_model = HuggingFaceBgeEmbeddings(
      model_name=self.embed_model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
    )

    if self.embed_model is not None:
        # PGVECTOR representation for embedding uses environmental variables to set the right database 
        os.environ['PGVECTOR_CONNECTION_STRING'] = "postgresql+psycopg2://%s:%s@%s:5432/%s"%(os.environ['POSTGRES_USER'], os.environ['POSTGRES_PASSWORD'], os.environ['POSTGRES_HOST'], self.name)
        self.embed_expression_list([])

  def start_session(self):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    return self.session

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Add Collections, Expressions, Items, and Fragments 
  # by running queries on external databases
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  #def add_corpus_from_epmc(self, qt, qt2, 
  #                         sections=['paper_title', 'ABSTRACT'], 
  #                         sections2=['paper_title', 'ABSTRACT'], 
  #                         page_size=1000):
  #  """Adds corpora based on coupled QueryTranslator objects."""
  #  
  #  if self.session is None:
  #    session_class = sessionmaker(bind=self.engine)
  #    self.session = session_class()
  #  
  #  if self.session.query(exists().where(InformationResource.name=='EPMC')).scalar():
  #    info_resource = self.session.query(InformationResource) \
  #        .filter(InformationResource.name=='EPMC').first()
  #  else:
  #    info_resource = InformationResource(id='skem:EPMC', 
  #                                        iri=['skem:EPMC'], 
  #                                        name='European PubMed Central', 
  #                                        type='skem:InformationResource',
  #                                        xref=['https://europepmc.org/'])
  #  
  #  (corpus_ids, epmc_queries) = qt.generate_queries(QueryType.epmc, sections=sections)
  #  if qt2:
  #    (subset_ids, epmc_subset_queries) = qt2.generate_queries(QueryType.epmc, sections=sections2)
  #  else: 
  #    (subset_ids, epmc_subset_queries) = ([0],[''])
  #  for (i, q) in zip(corpus_ids, epmc_queries):
  #    for (j, sq) in zip(subset_ids, epmc_subset_queries):
  #      query = q
  #      corpus_id = str(i)
  #      corpus_name = qt.df.loc[qt.df['ID']==i][qt.name_col].values[0]
  #      if query is None or query=='nan' or len(query)==0: 
  #        continue
  #      if len(sq) > 0 and sq != 'nan':
  #        query = '(%s) AND (%s)'%(q, sq)
  #        if j is not None:
  #          corpus_id = str(i)+'.'+str(j)
  #          corpus_name2 = qt2.df.loc[qt2.df['ID']==j][qt2.name_col].values[0]
  #          corpus_name = corpus_name + '/'+ corpus_name2
  #      try:
  #        self.add_corpus_from_epmc_query(corpus_id, corpus_name, query)
  #      except Exception as ex:
  #        print(ex)
  #        print('skipping %s'%(corpus_name) )
  #        continue
  #  self.session.commit()
  
  def add_corpus_from_epmc_query(self, 
                                 corpus_id, 
                                 corpus_name, 
                                 query, 
                                 commit_this=True,
                                 page_size=1000):
    """Adds corpus to database based on terms constructed in a direct EPMC query."""
    
    # does this collection already exist?  
    corpus_id = str(corpus_id)
    all_existing_query = self.session.query(SKC).filter(SKC.id==corpus_id)
    
    corpus = None
    for c in all_existing_query.all():
      corpus = c
    if corpus is None:      
      corpus = ScientificKnowledgeCollection(id=corpus_id,
                                           type='skem:ScientificKnowledgeCollection',
                                           name=corpus_name,
                                           has_members=[])
    self.session.add(corpus)

    epmcq = EuroPMCQuery()
    numFound, pubs = epmcq.run_empc_query(query, page_size=page_size)
    for p in tqdm(pubs):
      p_id = str(p.id)
      p_check = self.session.query(SKE) \
          .filter(SKE.id==p_id).first()
      if p_check is not None:
        p = p_check
      corpus.has_members.append(p)
      p.member_of.append(corpus)
      for item in p.has_representation:
        for f in item.has_part:
          #f.content = '\n'.join(self.sent_detector.tokenize(f.content))
          f.part_of = item.id
          self.session.add(f)
        item.represented_by = p.id
        self.session.add(item)
      self.session.add(p)
    
    if commit_this:
      self.session.commit()

    return corpus

  def check_query_terms(self, qt, qt2=None, pubmed_api_key=''):
    pmq = ESearchQuery(api_key=pubmed_api_key)
    terms = set()
    for t in qt.terms2id.keys():
        terms.add(t)
    if qt2 is not None:
        for t2 in qt2.terms2id.keys():
            terms.add(t2)
    check_table = {} 
    for t in tqdm(terms):
        (is_ok, t2, c) = pmq._check_query_phrase(t)
        check_table[t] = (is_ok, c)
    return check_table

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Add corpus based on a list of dois using OpenAlex
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def add_collection_from_dois_using_openalex(self, corpus_id, corpus_name, dois, commit_this=True):
    
    # does this collection already exist?  
    corpus_id = str(corpus_id)
    all_existing_query = self.session.query(SKC).filter(SKC.id==corpus_id)
    
    corpus = None
    for c in all_existing_query.all():
      corpus = c
    if corpus is None:      
      corpus = ScientificKnowledgeCollection(id=corpus_id,
                                           type='skem:ScientificKnowledgeCollection',
                                           name=corpus_name,
                                           has_members=[])
    self.session.add(corpus)

    papers_to_index = []
    for doi in tqdm(dois):
        p = load_paper_from_openalex(doi)
        if p is None:
          continue
        p_id = str(p.id)
        p_check = self.session.query(SKE) \
            .filter(SKE.id==p_id).first()
        if p_check is not None:
          p = p_check

        self.session.add(p)
        corpus.has_members.append(p)
        p.member_of.append(corpus)
        for item in p.has_representation:
            for f in item.has_part:
                #f.content = '\n'.join(self.sent_detector.tokenize(f.content))
                f.part_of = item.id
                self.session.add(f)
            item.represented_by = p.id
            self.session.add(item)
        papers_to_index.append(p)
        self.session.flush()

    self.embed_expression_list(papers_to_index)

    if commit_this:
      self.session.commit()

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Manipulating + Combining Collections
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def rename_collection(self, collection_id, new_collection_name, commit_this=True):
    """Renames a collection."""
    
    c = self.session.query(SKC).filter(SKC.id==collection_id).first()
    if c is None:
      raise Exception('Collection does not exist')
    c.name = new_collection_name
    if commit_this:
      self.session.commit()
    return c

  def create_new_collection_from_intersection(self, collection_id, collection_name, c1_id, c2_id, commit_this=True):
    """Creates a new collection based on the intersection of two existing collections."""
    
    c1 = self.session.query(SKC).filter(SKC.id==c1_id).first()
    c2 = self.session.query(SKC).filter(SKC.id==c2_id).first()
    
    if c1 is None or c2 is None:
      raise Exception('One or both of the collections do not exist')
    
    c1_items = set([m.id for m in c1.has_members])
    c2_items = set([m.id for m in c2.has_members])
    
    intersection = c1_items.intersection(c2_items)
    
    new_collection = ScientificKnowledgeCollection(id=collection_id,
                                                   type='skem:ScientificKnowledgeCollection',
                                                   name=collection_name,
                                                   has_members=[])
    self.session.add(new_collection)
    
    for i in intersection:
      item = self.session.query(SKE).filter(SKE.id==i).first()
      new_collection.has_members.append(item)
      item.member_of.append(new_collection)
    
    if commit_this:
      self.session.commit()

    return new_collection

  def create_new_collection_from_sample(self, collection_id, collection_name, c_id, count, subtypes=[], commit_this=True):
    """Creates a new collection based on the intersection of two existing collections."""
    
    q = self.session.query(SKE) \
      .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
      .filter(SKC_HM.has_members_id==SKE.id) \
      .filter(SKC.id==c_id)
    
    if len(subtypes)>0:
      q = q.filter(SKE.type.in_(subtypes))

    items = q.all()
    sample = random.sample(items, count)
        
    new_collection = ScientificKnowledgeCollection(id=collection_id,
                                                   type='skem:ScientificKnowledgeCollection',
                                                   name=collection_name,
                                                   has_members=[])
    self.session.add(new_collection)
    
    for e in sample:
      new_collection.has_members.append(e)
      e.member_of.append(new_collection)
    
    if commit_this:
      self.session.commit()

    return new_collection

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Deleting Collections, Expressions, Items, and Fragments.
  #     Need to adjust SQLAlchemy methods to permit cascades.
  #     This is more precise and careful (but need shortcuts)
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def delete_collection(self, collection_id, commit_this=True):    
    """Deletes the specified collection from the database, leave all expressions and items intact"""  

    if self.session.query(exists().where(SKC.id==collection_id)).scalar():

      # remove collection - expressions relations
      q2 = self.session.query(SKC_HM) \
          .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
          .filter(SKC.id==collection_id)
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()

      # remove expression -> collection relations
      q3 = self.session.query(SKE_MO) \
          .filter(SKE_MO.member_of_id==collection_id)
      for cp in q3.all():
        self.session.delete(cp)
        self.session.flush()

      q4 = self.session.query(SKC) \
          .filter(SKC.id==collection_id)
      for c in q4.all():
        self.session.delete(c)
        self.session.flush()

      q5 = self.session.query(SKC_HN, N) \
        .filter(SKC_HN.ScientificKnowledgeCollection_id==collection_id) \
        .filter(SKC_HN.has_notes_id==N.id)
      for hn, n in q5.all():
        self.session.delete(hn)
        self.session.delete(n)
        self.session.flush()

      q6 = self.session.query(NoteIsAbout) \
        .filter(NoteIsAbout.Note_id.like('skc:'+collection_id+'.'))
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Collection does not exist')

  def delete_expression(self, expression_id, commit_this=True):    
    """Deletes the specified collection from the database, leave all expressions and items intact"""  

    if self.session.query(exists().where(SKE.id==expression_id)).scalar():

      q = self.session.query(SKI) \
          .filter(SKE_HR.ScientificKnowledgeExpression_id==expression_id) \
          .filter(SKE_HR.has_representation_id==SKI.id)
      for item in q.all():
        self.delete_item(item.id, commit_this=False)
        self.session.flush()

      # remove collection - expressions relations
      q2 = self.session.query(SKC_HM) \
          .filter(SKC_HM.has_members_id==expression_id)
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()

      # remove expression -> collection relations
      q3 = self.session.query(SKE_MO) \
          .filter(SKE_MO.ScientificKnowledgeExpression_id==expression_id)
      for cp in q3.all():
        self.session.delete(cp)
        self.session.flush()

      q5 = self.session.query(SKE_HN, N) \
        .filter(SKE_HN.ScientificKnowledgeExpression_id==expression_id) \
        .filter(SKC_HN.has_notes_id==N.id)
      for hn, n in q5.all():
        self.session.delete(hn)
        self.session.delete(n)
        self.session.flush()

      q6 = self.session.query(NoteIsAbout) \
        .filter(NoteIsAbout.Note_id.like('ske:'+expression_id+'.'))
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      q7 = self.session.query(SKE_IRI) \
        .filter(SKE_IRI.ScientificKnowledgeExpression_id==expression_id)
      for iri in q7.all():
        self.session.delete(iri)
        self.session.flush()

      q8 = self.session.query(SKE_XREF) \
        .filter(SKE_XREF.ScientificKnowledgeExpression_id==expression_id)
      for xref in q8.all():
        self.session.delete(xref)
        self.session.flush()

      q4 = self.session.query(SKE) \
          .filter(SKE.id==expression_id)
      for c in q4.all():
        self.session.delete(c)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Collection does not exist')

  def delete_item(self, item_id, commit_this=True):    
    """Deletes the specified item from the database, cascade the deletion to all attached fragments and notes."""  

    if self.session.query(exists().where(SKI.id==item_id)).scalar():

      q = self.session.query(SKF) \
          .filter(SKI_HP.ScientificKnowledgeItem_id==item_id) \
          .filter(SKI_HP.has_part_id==SKF.id)
      for fragment in q.all():
        self.delete_fragment(fragment.id, commit_this=False)
        self.session.flush()

      # remove expression -> item relations
      q2 = self.session.query(SKE_HR) \
          .filter(SKE_HR.has_representation_id==item_id)
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()

      q4 = self.session.query(SKI) \
          .filter(SKI.id==item_id)
      for c in q4.all():
        self.session.delete(c)
        self.session.flush()

      q5 = self.session.query(SKI_HN, N) \
        .filter(SKI_HN.ScientificKnowledgeItem_id==item_id) \
        .filter(SKI_HN.has_notes_id==N.id)
      for hn, n in q5.all():
        self.session.delete(hn)
        self.session.delete(n)
        self.session.flush()

      q6 = self.session.query(NoteIsAbout) \
        .filter(NoteIsAbout.Note_id.like('ski:'+item_id+'.'))
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Collection does not exist')

  def delete_fragment(self, fragment_id, commit_this=True):    
    """Deletes the specified fragment from the database, cascade the deletion to all associated notes."""  

    if self.session.query(exists().where(SKF.id==fragment_id)).scalar():

      # remove item -> fragment relations
      q2 = self.session.query(SKI_HP) \
          .filter(SKI_HP.has_part_id==fragment_id)
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()

      q4 = self.session.query(SKF) \
          .filter(SKF.id==fragment_id)
      for c in q4.all():
        self.session.delete(c)
        self.session.flush()

      q5 = self.session.query(SKF_HN, N) \
        .filter(SKF_HN.ScientificKnowledgeFragment_id==fragment_id) \
        .filter(SKF_HN.has_notes_id==N.id)
      for hn, n in q5.all():
        self.session.delete(hn)
        self.session.delete(n)
        self.session.flush()

      q6 = self.session.query(NoteIsAbout) \
        .filter(NoteIsAbout.Note_id.like('skf:'+fragment_id+'.'))
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Collection does not exist')  

  def delete_note(self, note_id, commit_this=True):    
    """Deletes the specified note from the database, cascade the deletion to all associated entities."""  

    if self.session.query(exists().where(Note.id==note_id)).scalar():

      # remove item -> fragment relations
      q1 = self.session.query(NoteIsAbout) \
          .filter(NoteIsAbout.Note_id==note_id) 
      for cp in q1.all():
        self.session.delete(cp)
        self.session.flush()

      q2 = self.session.query(SKF_HN) \
          .filter(SKF_HN.has_notes_id==note_id) 
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()
 
      q3 = self.session.query(SKC_HN) \
          .filter(SKC_HN.has_notes_id==note_id) 
      for cp in q3.all():
        self.session.delete(cp)
        self.session.flush()

      q4 = self.session.query(SKE_HN) \
          .filter(SKE_HN.has_notes_id==note_id) 
      for cp in q4.all():
        self.session.delete(cp)
        self.session.flush()

      q5 = self.session.query(SKI_HN) \
          .filter(SKI_HN.has_notes_id==note_id) 
      for cp in q5.all():
        self.session.delete(cp)
        self.session.flush()

      q6 = self.session.query(N_HN) \
          .filter(N_HN.has_notes_id==note_id) 
      for cp in q6.all():
        self.session.delete(cp)
        self.session.flush()

      q6 = self.session.query(Note) \
        .filter(Note.id==note_id)
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Note does not exist')  

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Analyze the structure / composition of a collection
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def build_full_text_report_on_collection(self, collection_id):
    q = self.session.query(SKE) \
        .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
        .filter(SKC_HM.has_members_id==SKE.id) \
        .filter(SKC.id==str(collection_id)) \
        .order_by(desc(SKE.publication_date))
    return pd.DataFrame([self.check_full_text(e) for e in tqdm(q.all())])    

  def build_item_type_pivot_table(self):
    q = self.session.query(SKC.name, SKE.id, SKI.type) \
        .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
        .filter(SKC_HM.has_members_id==SKE.id) \
        .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id==SKI.id) 
    df = pd.DataFrame(q.all(), columns=['collection', 'doi', 'type'])    
    return df.pivot_table(index='collection', columns='type', values='doi', aggfunc=lambda x: len(x.unique()))

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Add full text elements to database based on dowloaded files
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def add_full_text_for_expression(self, e):
    """Adds Full Text Item data to given Expression 
    if full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = re.sub('doi:','',e.id)
    
    # Check local directory for files
    base_file_path = '%s%s/ft/'%(self.loc, self.name)
    nxml_file_path = base_file_path+doi+'.nxml'
    pdf_file_path = base_file_path+doi+'.pdf'
    html_file_path = base_file_path+doi+'.html'

    self.add_full_text_for_expression_nxml(e, nxml_file_path)
    self.add_full_text_for_expression_pdf(e, pdf_file_path)
    self.add_full_text_for_expression_html(e, html_file_path)
    self.session.commit() 

  def add_full_text_for_expression_nxml(self, e, nxml_file_path, update_existing=False):
    """Adds Full Text Item data to expression if *.nxml full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = re.sub('doi:','',e.id)

    q = self.session.query(SKI) \
        .filter(SKE_HR.ScientificKnowledgeExpression_id == e.id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(or_(SKI.type == 'JATSFullText'))
    if update_existing is False:
      for i in q.all():
        return
    else:
      for i in q.all():
        self.delete_item(i.id, commit_this=False)
        self.session.commit()

    if os.path.exists(nxml_file_path):
      with open(nxml_file_path, 'r') as f:
        xml = f.read()
        try:
          d = NxmlDoc(doi, xml)
        except Exception as ex:
          return
        doc_text = d.text 
        if 'body' in [so.element.tag for so in d.standoffs] is False:
          return
        ski_type = 'JATSFullText'
                
        ski_id = str(uuid.uuid4().hex)[:10]
        ski = ScientificKnowledgeItem(id=ski_id, 
                                      content=doc_text,
                                      xref=['file:'+nxml_file_path],
                                      type=ski_type,)
        e.has_representation.append(ski)
        self.session.add(ski)
        self.session.flush()
        self.add_sections_as_fragments_from_nxmldoc(ski, d)
        self.session.commit()

  def add_sections_as_fragments_from_nxmldoc(self, item, d):
    """Adds fragments from an NxmlDoc object to a ScientificKnowledgeItem object."""

    # ['PMID', 'PARAGRAPH_ID', 'TAG', 'TAG_TREE', 'OFFSET', 'LENGTH', 'FIG_REF', 'PLAIN_TEXT'
    fragments = []
    df = d.build_enhanced_document_dataframe()
    # df = df[df.SECTION_TREE!='']
    # Can search for a substring in top section headers
    # df3 = df[df.TOP_SECTION.str.contains('method', case=False)]

    try: 
      df3 = df[df.TOP_SECTION!='']
      df4 = df3.groupby('SECTION_TREE').agg({'OFFSET': 'min', 'LENGTH': 'sum', 'PLAIN_TEXT':lambda x: '\n'.join(x)}).sort_values('OFFSET')
      df4 = df4.reset_index(drop=False)
    
      if df4 is None:
        return
    
      for i, tup in df4.iterrows():      
        #sentence_split_text = '\n'.join(self.sent_detector.tokenize(tup.PLAIN_TEXT))
        sentence_split_text = tup.PLAIN_TEXT
        fragment = ScientificKnowledgeFragment(id=item.id[:10]+'.'+str(i), 
                                              type='section',
                                              offset=tup.OFFSET,
                                              length=tup.LENGTH,
                                              name=tup.SECTION_TREE,
                                              content=sentence_split_text)
        self.session.add(fragment)
        self.session.flush()
        item.has_part.append(fragment)
        fragment.part_of = item.id
        fragments.append(fragment)
        self.session.flush() 
      self.session.commit()
    except Exception as e:
      self.session.rollback()    
      
  def add_full_text_for_expression_pdf(self, e, pdf_file_path, update_existing=False):
    """Adds Full Text Item data to expression if *.pdf full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = re.sub('doi:','',e.id)

    q = self.session.query(SKI) \
        .filter(SKE_HR.ScientificKnowledgeExpression_id == e.id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(or_(SKI.type == 'PDFFullText'))
    if update_existing is False:
      for i in q.all():
        return
    else:
      for i in q.all():
        self.delete_item(i.id, commit_this=False)
        self.session.commit()

    if os.path.exists(pdf_file_path):
      if os.environ.get('HURIDOCS_NETWORK'):
        loader = HuridocsPDFLoader(pdf_file_path, host=os.environ.get('HURIDOCS_NETWORK'))
      else:   
        loader = HuridocsPDFLoader(pdf_file_path)
      #loader = LAPDFBlockLoader(pdf_file_path)
      docs = loader.load(curi_id='doi:'+doi)
      skfs = []
      doc_text = ''
      ski_type = 'PDFFullText'

      ski_id = str(uuid.uuid4().hex)[:10]
      for i, d in enumerate(docs):
        #sentence_split_text = '\n'.join(self.sent_detector.tokenize(d.page_content))
        sentence_split_text = d.page_content

        skf = ScientificKnowledgeFragment(id=ski_id+'.'+str(i), 
                                             type='section',
                                             offset=len(doc_text),
                                             length=len(sentence_split_text),
                                             name=sentence_split_text.split('\n')[0],
                                             content=sentence_split_text)
        if len(doc_text) > 0:
          doc_text += '\n\n'
        doc_text += sentence_split_text
        self.session.add(skf)
        self.session.flush()
        skfs.append(skf)
      ski = ScientificKnowledgeItem(id=ski_id, 
                                    content=doc_text,
                                    xref=['file:'+pdf_file_path],
                                    type=ski_type,)
      e.has_representation.append(ski)
      self.session.add(ski)
      self.session.flush()
      for skf in skfs:
        ski.has_part.append(skf)
        skf.part_of = ski.id
        self.session.flush()
  
  def add_full_text_for_expression_html(self, e, html_file_path, update_existing=False):
    """Adds Full Text Item data to expression if *.html full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = re.sub('doi:','',e.id)

    q = self.session.query(SKI) \
        .filter(SKE_HR.ScientificKnowledgeExpression_id == e.id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(or_(SKI.type == 'HTMLFullText'))
    if update_existing is False:
      for i in q.all():
        return
    else:
      for i in q.all():
        self.delete_item(i.id, commit_this=False)
        self.session.commit()

    if os.path.exists(html_file_path):
      loader = TrafilaturaSectionLoader(html_file_path)
      #loader = LAPDFBlockLoader(pdf_file_path)
      try: 
        docs = loader.load()
      except Exception as ex:
        print('Error loading HTML file: %s'%(html_file_path))
        return
      skfs = []
      doc_text = ''
      ski_type = 'HTMLFullText'

      ski_id = str(uuid.uuid4().hex)[:10]
      for i, d in enumerate(docs):
        #sentence_split_text = '\n'.join(self.sent_detector.tokenize(d.page_content))
        sentence_split_text = d.page_content

        skf = ScientificKnowledgeFragment(id=ski_id+'.'+str(i), 
                                             type='section',
                                             offset=len(doc_text),
                                             length=len(sentence_split_text),
                                             name=sentence_split_text.split('\n')[0],
                                             content=sentence_split_text)
        if len(doc_text) > 0:
          doc_text += '\n\n'
        doc_text += sentence_split_text
        self.session.add(skf)
        self.session.flush()
        skfs.append(skf)
      ski = ScientificKnowledgeItem(id=ski_id, 
                                    content=doc_text,
                                    xref=['file:'+html_file_path],
                                    type=ski_type,)
      e.has_representation.append(ski)
      self.session.add(ski)
      self.session.flush()
      for skf in skfs:
        ski.has_part.append(skf)
        skf.part_of = ski.id
        self.session.flush()
        
  def add_full_text_for_collection(self, collection_id, 
                                   get_nxml=True, get_pdf=True, get_html=True):
    for e in self.list_expressions(collection_id=collection_id):
      self.add_full_text_for_expression(e, get_nxml, get_pdf, get_html)  
  
  def check_full_text(self, e):
      """Checks if full text is available and readable for a given expression."""

      doi = re.sub('doi:','',e.id)
      file_path = self.loc + self.name + '/ft/'
      nxml_file_path = file_path+doi+'.nxml'
      pdf_file_path = file_path+doi+'.pdf'
      html_file_path = file_path+doi+'.html'
      
      nxml_item_present = False
      q = self.session.query(SKI) \
          .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
          .filter(SKE_HR.has_representation_id==SKI.id) \
          .filter(SKI.type == 'JATSFullText') \
          .filter(SKE.id==e.id)
      if q.count() > 0:
          nxml_item_present = True
      nxml_present = os.path.exists(nxml_file_path)
      nxml_has_body = False
      if os.path.exists(nxml_file_path):
          with open(nxml_file_path, 'r') as file:
              soup = BeautifulSoup(file, "lxml-xml")
              if soup.body is not None:
                  nxml_has_body = True
      
      pdf_item_present = False
      q2 = self.session.query(SKI) \
          .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
          .filter(SKE_HR.has_representation_id==SKI.id) \
          .filter(SKI.type == 'PDFFullText') \
          .filter(SKE.id==e.id)
      if q2.count() > 0:
          pdf_item_present = True
      pdf_present = os.path.exists(pdf_file_path)
      pdf_readable = False
      if pdf_present:
          try: 
              with fitz.open(pdf_file_path) as doc:
                  pdf_readable = True
                  #print('PDF: %s, Page Count: %d'%(doi, doc.page_count))
          except Exception as ex:
              pdf_readable = False
                      
      if os.path.exists(html_file_path):
          html_present = True
      else:
          html_present = False

      html_readable = False
      if html_present:
        loader = TrafilaturaSectionLoader(html_file_path)
        try: 
          docs = loader.load()
          html_readable = True
        except Exception as ex:
          html_readable = False

      prior_attempts = False
      for n in self.read_notes_about_x(e):
        if 'download' in n.name:
          prior_attempts = True
          break

      report = {'doi': doi, 
                'nxml_item_present': nxml_item_present,
                'nxml_present': nxml_present,
                'nxml_has_body': nxml_has_body,
                'pdf_item_present': pdf_item_present,
                'pdf_present': pdf_present,
                'pdf_readable': pdf_readable,
                'html_present': html_present,
                'html_readable': html_readable,
                'prior_attempts': prior_attempts}
      
      return report

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Read and Write Notes for Entities
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def write_note_about_x(self, x, note_name, note_content, note_format, note_type, commit_this=False):
    """Writes a note about an entity in the database."""
    note_id = str(uuid.uuid4().hex)[:10]
    creation_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
    note = Note(id=note_id, 
             name=note_name, 
             creation_date=creation_date,
             format=note_format,
             type=note_type, 
             content=note_content)
    self.session.add(note)
    x.has_notes.append(note)
    #note.is_about.append(x)
    self.session.flush()
    if commit_this:
      self.session.commit()
    return note
  
  def read_notes_about_x(self, x):
    """Reads notes about an entity in the database."""
    q = self.session.query(N) \
        .filter(N.id == NIA.Note_id) \
        .filter(NIA.is_about_id == x.id)
    for n in q.all():
      yield(n)

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Quick reports
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def report_collection_composition(self):
    """Returns a DataFrame with the composition of all collections in the database."""
    q = self.session.query(SKC.id, SKC.name, SKE.id, SKI.type) \
            .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
            .filter(SKC_HM.has_members_id==SKE.id) \
            .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id==SKI.id) 
    df = pd.DataFrame(q.all(), columns=['id', 'collection name', 'doi', 'item type'])
    piv_df = df.pivot_table(index=['id', 'collection name'], columns='item type', values='doi', aggfunc=lambda x: len(x.unique())).fillna(0)
    return piv_df

  def report_non_full_text_for_collection(self, collection_id):
    """Returns a DataFrame listing all papers without PDF or JATS full text versions in the database."""
    q = self.session.query(SKC.id, SKE.id, SKI.type) \
            .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
            .filter(SKC_HM.has_members_id==SKE.id) \
            .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id==SKI.id) \
            .filter(SKC.id==str(collection_id))
    df = pd.DataFrame(q.all(), columns=['collection_id', 'doi', 'item type'])
    piv_df = df.pivot_table(index=['doi'], 
                        columns='item type', values='collection_id', 
                        aggfunc=lambda x: len(x.unique()))
    piv_df.fillna(0, inplace=True)
    piv_df = piv_df.reset_index(drop=False).sort_values(by='doi')
    return piv_df[(piv_df['JATSFullText']==0.0) & (piv_df['PDFFullText']==0.0)]

  def report_all_notes_by_type_name(self, note_type, name_pattern):
    """Returns a DataFrame listing all note contents with full text versions in the database."""
    q = self.session.query(N) \
      .filter(N.id == NIA.Note_id) \
      .filter(N.type == note_type) \
      .filter(N.name.like(name_pattern)) 
    l = []
    for n in q.all():
        tup = json.loads(n.content)
        tup = json.loads(n.provenance)
        tup['note_id'] = n.name
        l.append(tup)
    report_df = pd.DataFrame(l).set_index('note_id')
    return report_df

  def create_zip_archive_of_full_text_files(self, collection_id, zip_file_path):
    """Creates a zip archive of all full text files for a given collection."""
    file_path = self.loc + self.name + '/ft/'
    with ZipFile(zip_file_path, 'w') as myzip:
      for e in self.list_expressions(collection_id=collection_id):
        doi = re.sub('doi:','',e.id)
        nxml_file_path = file_path+doi+'.nxml'
        pdf_file_path = file_path+doi+'.pdf'
        html_file_path = file_path+doi+'.html'
        if os.path.exists(nxml_file_path):
          myzip.write(nxml_file_path, arcname=doi+'.nxml')
        if os.path.exists(pdf_file_path):
          myzip.write(pdf_file_path, arcname=doi+'.pdf')
        if os.path.exists(html_file_path):
          myzip.write(html_file_path, arcname=doi+'.html')
          
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Query Collections, Expressions, Items, and Fragments
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def get_expression_by_doi(self, doi):
    """Returns a ScientificKnowledgeExpression object for a given DOI."""
    self.start_session()
    query = self.session.query(SKE) \
        .filter(SKE.id==SKE_XREF.ScientificKnowledgeExpression_id) \
        .filter(SKE_XREF.xref=='doi:'+doi)
    ske = query.first()
    return ske

  def list_items_for_expression(self, doi):
    """Returns all items for an expression for a given DOI."""
    self.start_session()
    if doi[:4] != 'doi:':
      doi = 'doi:'+doi
    query = self.session.query(SKE, SKI) \
          .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
          .filter(SKE_HR.has_representation_id == SKI.id) \
          .filter(SKE.id == doi)
    l_it = []
    for (ex, it) in query.all():
      l_it.append(it)
    return l_it

  def list_collections(self, search_term=None):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if search_term:
      q = (self.session.query(SKC) 
          .filter(SKC.name == search_term))
    else:
      q = self.session.query(SKC)
    for c in q.all():
      yield(c)

  def list_expressions(self, collection_id=None, search_term=None): 
    '''Lists expressions in database. 
    Including the `collection_id` parameter limits the returned list to the specified collection. 
    Including the `search_term` parameter searches the citation string for the specified term.'''

    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if collection_id:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(SKE) 
            .filter(SKC_HM.ScientificKnowledgeCollection_id==collection_id)
            .filter(SKC_HM.has_members_id == SKE.id)
            .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id)
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.content.like(search))
          )
      else:
         q = (self.session.query(SKE) 
            .filter(SKC.id == SKC_HM.ScientificKnowledgeCollection_id)
            .filter(SKC_HM.has_members_id == SKE.id)
            .filter(SKC.id == collection_id)
          )
    else:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(SKE) 
            .filter(SKC_HM.has_members_id == SKE.id)
            .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id)
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.content.like(search))
          )
      else:
        q = self.session.query(SKE)
      
    for c in q.all():
      yield(c)

  def list_notes_for_fragments_in_paper(self, run_name, paper_id, item_type='FullTextPaper'):
    '''returns notes of a specific type associated with fragments from a given paper .'''
    q1 = self.session.query(SKI) \
            .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id == SKI.id) \
            .filter(SKI.type == item_type) \
            .filter(SKE.id.like('%'+str(paper_id)+'%')) 
    i = q1.first()
    if i is None:
      return []
    for f in i.has_part:
      for n in f.has_notes:
        if n.name == run_name:
          yield(n)
         
  def list_fragments_for_paper(self, paper_id, item_type, fragment_types=['title', 'abstract']):
    '''Loads fragments from a given paper sections of a specified paper from the local database.'''
    q1 = self.session.query(SKI) \
            .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id == SKI.id) \
            .filter(SKI.type == item_type) \
            .filter(SKE.id.like('%'+str(paper_id)+'%')) 
    i = q1.first()
    if i is None:
      return []
    fragments = []
    for f in i.has_part:
      if f.type in fragment_types:
        fragments.append(f)
    return sorted(fragments, key=lambda f: f.offset)
  
  def list_fragments(self, expression_id=None, search_term=None):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if expression_id:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(SKF) 
            .filter(SKE_HR.ScientificKnowledgeExpression_id == expression_id)
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id)
            .filter(SKI_HP.has_part_id == SKF.id)
            .filter(SKF.content.ilike('%'+search+'%'))
          )
      else:
        q = (self.session.query(SKF) 
            .filter(SKE_HR.ScientificKnowledgeExpression_id == expression_id)
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id)
            .filter(SKI_HP.has_part_id == SKF.id)
          )
    else:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(SKF) 
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id)
            .filter(SKI_HP.has_part_id == SKF.id)
            .filter(SKF.content.ilike('%'+search+'%'))
          )
      else:
        q = self.session.query(SKF)
    for c in q.all():
      yield(c)

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Build RAG Indexes of documents
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
  def embed_expression_list(self, explist):
    '''Runs through the list of expressions, generates embeddings, and stores them in the database'''

    texts = []
    metadatas = []
    
    for e in tqdm(explist):
      coll_ids = ','.join([c.id for c in e.member_of])

      t, a = self.list_fragments_for_paper(e.id, 'CitationRecord')
      i = t.part_of
      tiab_text = t.content+'\n\n'+a.content

      if len(tiab_text) > 0:
        texts.append(tiab_text)
        metadatas.append({'c_ids': coll_ids, \
                                  'e_id': e.id, \
                                  'e_type': e.type, \
                                  'i_id': t.part_of, \
                                  'i_type': 'CitationRecord', \
                                  'f_id': t.id, \
                                  'f_type': t.type+','+a.type, \
                                  'citation': e.content})

      jats_frgs = self.list_fragments_for_paper(e.id, 'JATSFullText', ['section'])
      for f in jats_frgs:
        texts.append(f.content)
        metadatas.append({'c_ids': coll_ids, \
                               'e_id': e.id, \
                               'e_type': e.type, \
                               'i_id': f.part_of, \
                               'i_type': 'JATSFullText', \
                               'f_id': f.id, \
                               'citation': e.content})        

      pdf_frgs = self.list_fragments_for_paper(e.id, 'PDFFullText', ['section'])
      for f in pdf_frgs:
        texts.append(f.content)
        metadatas.append({'c_ids': coll_ids, \
                              'e_id': e.id, \
                              'e_type': e.type, \
                              'i_id': f.part_of, \
                              'i_type': 'PDFFullText', \
                              'f_id': f.id, \
                              'citation': e.content})

    # Use this when we want to index text on individual sentences       
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size = 3000, chunk_overlap=150)
    
    # build vector indexes from citation record     
    #tiab_docs = text_splitter.create_documents(tiab_texts, metadatas=tiab_metadatas)
    docs = []
    for t,m in zip(texts, metadatas):
      docs.append(Document(page_content=t, metadata=m))
    
    print('Indexing %d documents'%(len(docs)))
    db = PGVector.from_documents(
        embedding=self.embed_model,
        documents=docs,
        collection_name="ScienceKnowledgeItem"
    )

  def query_vectorindex(self, query_string, entity_type='ScienceKnowledgeItem', k=4):
    db2 = PGVector.from_existing_index(
      embedding=self.embed_model, 
      collection_name=entity_type) 
    docs = db2.similarity_search_with_score(query_string, k=k)
    return docs
  
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Retrieve and manipulate raw vectors
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def list_unindexed_skes(self, collection_id):
    query = """
        SELECT DISTINCT ske.id
        FROM "ScientificKnowledgeExpression" as ske
            JOIN "ScientificKnowledgeCollection_has_members" as skc_hm on ske.id = skc_hm.has_members_id
            LEFT JOIN langchain_pg_embedding as emb on emb.cmetadata->>'e_id' = ske.id
        WHERE emb.cmetadata->>'e_id' IS NULL AND
            skc_hm."ScientificKnowledgeCollection_id"='{}';
        """.format(collection_id)
    skes = []
    for ske_id in self.session.execute(text(query)).fetchall():
      skes.extend(self.session.query(SKE).filter(SKE.id==ske_id[0]).all())
    return skes
  
  def list_embedded_skes_for_collection_by_fragment_type(self, collection_id, f_type='title,abstract'):
    query = """
        SELECT DISTINCT ske.id, emb.uid, emb.cmetadata->>'f_id'
        FROM langchain_pg_embedding as emb, 
            "ScientificKnowledgeExpression" as ske,
            "ScientificKnowledgeCollection_has_members" as skc_hm
        WHERE emb.cmetadata->>'f_type' = '{}}' AND
            emb.cmetadata->>'e_id' = ske.id AND 
            ske.id = skc_hm.has_members_id AND
            skc_hm."ScientificKnowledgeCollection_id"='{}';
        """.format(f_type, collection_id)
    rag_data = self.session.execute(text(query)).fetchall()
    return rag_data

  def delete_repeated_embeddings_for_collection_by_fragment_type(self, collection_id, f_type='title,abstract'):
    rag_data = self.list_embedded_skes_for_collection_by_fragment_type(collection_id, f_type=f_type)
    checked_dois = []
    to_remove = []
    for tup in rag_data:
        if tup[2] not in checked_dois:
            checked_dois.append(tup[2])
        else:
            to_remove.append(tup[1])
    for Uuid in to_remove:
      self.session.execute(text('DELETE FROM langchain_pg_embedding WHERE uuid=:uuid'), {'uuid':Uuid})
    self.session.commit()

  def retrieve_vectors_for_collection_by_item_type(self, collection_id, i_type='CitationRecord'):
    query = """
        SELECT DISTINCT ske.id, emb.embedding 
        FROM langchain_pg_embedding as emb, 
            "ScientificKnowledgeExpression" as ske,
            "ScientificKnowledgeCollection_has_members" as skc_hm
        WHERE emb.cmetadata->>'i_type' = '{}}' AND
            emb.cmetadata->>'e_id' = ske.id AND 
            ske.id = skc_hm.has_members_id AND
            skc_hm."ScientificKnowledgeCollection_id"='{}';
        """.format(i_type, collection_id)
    rag_data = self.session.execute(text(query)).fetchall()
    return rag_data

  def retrieve_vectors_for_collection_by_fragment_type(self, collection_id, f_type='title,abstract'):
    query = """
        SELECT DISTINCT ske.id, ske.content, emb.embedding 
        FROM langchain_pg_embedding as emb, 
            "ScientificKnowledgeExpression" as ske,
            "ScientificKnowledgeCollection_has_members" as skc_hm
        WHERE emb.cmetadata->>'f_type' = '{}}' AND
            emb.cmetadata->>'e_id' = ske.id AND 
            ske.id = skc_hm.has_members_id AND
            skc_hm."ScientificKnowledgeCollection_id"='{}';
        """.format(f_type, collection_id)
    rag_data = self.session.execute(text(query)).fetchall()
    return rag_data

`=======`

In [ ]:
#| export 

class Ceifns_LiteratureDb(BaseModel):
  """This class runs a set of queries on external literature databases to build a local database of linked corpora and papers.

  Functionality includes:

    * Executes queries over European PMC 
    * Can run combinatorial sets of queries using a dataframe structure
        * Requires a column of queries expressed in boolean logic
        * Optional to define a secondary spreadsheet with a column of subqueries expressed in boolean logic
    * Has capability to run boolean logic over sources (currently only European PMC, but possibly others) 
    * Builds a local Postgresql database with tables for collections, expressions, items, fragments, and notes. 
    * Provides an API for querying the database and returning results as sqlAlchemy objects.
    * Permits user to download a local copy of full text papers in NXML(JATS), PDF, and HTML format.
  """
  loc: str = Field()
  name: str = Field()
  engine: Engine = Field(default=None, init=False)
  session: Session = Field(default=None, init=False)
  sent_detector: nltk.tokenize.punkt.PunktSentenceTokenizer = Field(default=None, init=False)
  #embed_model_name: str = Field(default='BAAI/bge-large-en-v1.5', init=False)
  #embed_model_device: str = Field(default='cpu', init=False)
  embed_model: Embeddings = Field(default=None, init=False)

  class Config:
    """Configuration for this pydantic object."""
    arbitrary_types_allowed = True

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    
    if self.loc[-1] != '/':
      self.loc = self.loc + '/' 
    if os.path.exists(self.loc) is False:
      os.makedirs(self.loc)

    db_dir = Path(self.loc + self.name)
    if db_dir.exists() is False:
      os.makedirs(db_dir)

    self.engine = create_engine("postgresql+psycopg2://%s:%s@%s:5432/%s"%(os.environ['POSTGRES_USER'], os.environ['POSTGRES_PASSWORD'], os.environ['POSTGRES_HOST'], self.name))

    self.start_session() # instantiate session 

    log_path = '%s%s_log.txt'%(self.loc, self.name)
    if os.path.exists(log_path) is False:
      Path(log_path).touch()
    
    self.sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

    #
    # hard coded embedding model at present.
    #
    #model_kwargs = {"device": self.embed_model_device}
    #encode_kwargs = {"normalize_embeddings": True}
    #self.embed_model = HuggingFaceBgeEmbeddings(
    #  model_name=self.embed_model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
    #)

    if self.embed_model is not None:
        # PGVECTOR representation for embedding uses environmental variables to set the right database 
        os.environ['PGVECTOR_CONNECTION_STRING'] = "postgresql+psycopg2://%s:%s@%s:5432/%s"%(os.environ['POSTGRES_USER'], os.environ['POSTGRES_PASSWORD'], os.environ['POSTGRES_HOST'], self.name)
        self.embed_expression_list([])

  def start_session(self):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    return self.session

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Add Collections, Expressions, Items, and Fragments 
  # by running queries on external databases
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  #def add_corpus_from_epmc(self, qt, qt2, 
  #                         sections=['paper_title', 'ABSTRACT'], 
  #                         sections2=['paper_title', 'ABSTRACT'], 
  #                         page_size=1000):
  #  """Adds corpora based on coupled QueryTranslator objects."""
  #  
  #  if self.session is None:
  #    session_class = sessionmaker(bind=self.engine)
  #    self.session = session_class()
  #  
  #  if self.session.query(exists().where(InformationResource.name=='EPMC')).scalar():
  #    info_resource = self.session.query(InformationResource) \
  #        .filter(InformationResource.name=='EPMC').first()
  #  else:
  #    info_resource = InformationResource(id='skem:EPMC', 
  #                                        iri=['skem:EPMC'], 
  #                                        name='European PubMed Central', 
  #                                        type='skem:InformationResource',
  #                                        xref=['https://europepmc.org/'])
  #  
  #  (corpus_ids, epmc_queries) = qt.generate_queries(QueryType.epmc, sections=sections)
  #  if qt2:
  #    (subset_ids, epmc_subset_queries) = qt2.generate_queries(QueryType.epmc, sections=sections2)
  #  else: 
  #    (subset_ids, epmc_subset_queries) = ([0],[''])
  #  for (i, q) in zip(corpus_ids, epmc_queries):
  #    for (j, sq) in zip(subset_ids, epmc_subset_queries):
  #      query = q
  #      corpus_id = str(i)
  #      corpus_name = qt.df.loc[qt.df['ID']==i][qt.name_col].values[0]
  #      if query is None or query=='nan' or len(query)==0: 
  #        continue
  #      if len(sq) > 0 and sq != 'nan':
  #        query = '(%s) AND (%s)'%(q, sq)
  #        if j is not None:
  #          corpus_id = str(i)+'.'+str(j)
  #          corpus_name2 = qt2.df.loc[qt2.df['ID']==j][qt2.name_col].values[0]
  #          corpus_name = corpus_name + '/'+ corpus_name2
  #      try:
  #        self.add_corpus_from_epmc_query(corpus_id, corpus_name, query)
  #      except Exception as ex:
  #        print(ex)
  #        print('skipping %s'%(corpus_name) )
  #        continue
  #  self.session.commit()
  
  def add_corpus_from_epmc_query(self, 
                                 corpus_id, 
                                 corpus_name, 
                                 query, 
                                 commit_this=True,
                                 page_size=1000):
    """Adds corpus to database based on terms constructed in a direct EPMC query."""
    
    # does this collection already exist?  
    corpus_id = str(corpus_id)
    all_existing_query = self.session.query(SKC).filter(SKC.id==corpus_id)
    
    corpus = None
    for c in all_existing_query.all():
      corpus = c
    if corpus is None:      
      corpus = ScientificKnowledgeCollection(id=corpus_id,
                                           type='skem:ScientificKnowledgeCollection',
                                           name=corpus_name,
                                           has_members=[])
    self.session.add(corpus)

    epmcq = EuroPMCQuery()
    numFound, pubs = epmcq.run_empc_query(query, page_size=page_size)
    for p in tqdm(pubs):
      p_id = str(p.id)
      p_check = self.session.query(SKE) \
          .filter(SKE.id==p_id).first()
      if p_check is not None:
        p = p_check
      corpus.has_members.append(p)
      p.member_of.append(corpus)
      for item in p.has_representation:
        for f in item.has_part:
          #f.content = '\n'.join(self.sent_detector.tokenize(f.content))
          f.part_of = item.id
          self.session.add(f)
        item.represented_by = p.id
        self.session.add(item)
      self.session.add(p)
    
    if commit_this:
      self.session.commit()

    return corpus

  def check_query_terms(self, qt, qt2=None, pubmed_api_key=''):
    pmq = ESearchQuery(api_key=pubmed_api_key)
    terms = set()
    for t in qt.terms2id.keys():
        terms.add(t)
    if qt2 is not None:
        for t2 in qt2.terms2id.keys():
            terms.add(t2)
    check_table = {} 
    for t in tqdm(terms):
        (is_ok, t2, c) = pmq._check_query_phrase(t)
        check_table[t] = (is_ok, c)
    return check_table

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Add corpus based on a list of dois using OpenAlex
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def add_collection_from_dois_using_openalex(self, corpus_id, corpus_name, dois, commit_this=True):
    
    # does this collection already exist?  
    corpus_id = str(corpus_id)
    all_existing_query = self.session.query(SKC).filter(SKC.id==corpus_id)
    
    corpus = None
    for c in all_existing_query.all():
      corpus = c
    if corpus is None:      
      corpus = ScientificKnowledgeCollection(id=corpus_id,
                                           type='skem:ScientificKnowledgeCollection',
                                           name=corpus_name,
                                           has_members=[])
    self.session.add(corpus)

    papers_to_index = []
    for doi in tqdm(dois):
        p = load_paper_from_openalex(doi)
        if p is None:
          continue
        p_id = str(p.id)
        p_check = self.session.query(SKE) \
            .filter(SKE.id==p_id).first()
        if p_check is not None:
          p = p_check

        self.session.add(p)
        corpus.has_members.append(p)
        p.member_of.append(corpus)
        for item in p.has_representation:
            for f in item.has_part:
                #f.content = '\n'.join(self.sent_detector.tokenize(f.content))
                f.part_of = item.id
                self.session.add(f)
            item.represented_by = p.id
            self.session.add(item)
        papers_to_index.append(p)
        self.session.flush()

    self.embed_expression_list(papers_to_index)

    if commit_this:
      self.session.commit()

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Manipulating + Combining Collections
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def rename_collection(self, collection_id, new_collection_name, commit_this=True):
    """Renames a collection."""
    
    c = self.session.query(SKC).filter(SKC.id==collection_id).first()
    if c is None:
      raise Exception('Collection does not exist')
    c.name = new_collection_name
    if commit_this:
      self.session.commit()
    return c

  def create_new_collection_from_intersection(self, collection_id, collection_name, c1_id, c2_id, commit_this=True):
    """Creates a new collection based on the intersection of two existing collections."""
    
    c1 = self.session.query(SKC).filter(SKC.id==c1_id).first()
    c2 = self.session.query(SKC).filter(SKC.id==c2_id).first()
    
    if c1 is None or c2 is None:
      raise Exception('One or both of the collections do not exist')
    
    c1_items = set([m.id for m in c1.has_members])
    c2_items = set([m.id for m in c2.has_members])
    
    intersection = c1_items.intersection(c2_items)
    
    new_collection = ScientificKnowledgeCollection(id=collection_id,
                                                   type='skem:ScientificKnowledgeCollection',
                                                   name=collection_name,
                                                   has_members=[])
    self.session.add(new_collection)
    
    for i in intersection:
      item = self.session.query(SKE).filter(SKE.id==i).first()
      new_collection.has_members.append(item)
      item.member_of.append(new_collection)
    
    if commit_this:
      self.session.commit()

    return new_collection

  def create_new_collection_from_sample(self, collection_id, collection_name, c_id, count, subtypes=[], commit_this=True):
    """Creates a new collection based on the intersection of two existing collections."""
    
    q = self.session.query(SKE) \
      .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
      .filter(SKC_HM.has_members_id==SKE.id) \
      .filter(SKC.id==c_id)
    
    if len(subtypes)>0:
      q = q.filter(SKE.type.in_(subtypes))

    items = q.all()
    sample = random.sample(items, count)
        
    new_collection = ScientificKnowledgeCollection(id=collection_id,
                                                   type='skem:ScientificKnowledgeCollection',
                                                   name=collection_name,
                                                   has_members=[])
    self.session.add(new_collection)
    
    for e in sample:
      new_collection.has_members.append(e)
      e.member_of.append(new_collection)
    
    if commit_this:
      self.session.commit()

    return new_collection

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Deleting Collections, Expressions, Items, and Fragments.
  #     Need to adjust SQLAlchemy methods to permit cascades.
  #     This is more precise and careful (but need shortcuts)
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def delete_collection(self, collection_id, commit_this=True):    
    """Deletes the specified collection from the database, leave all expressions and items intact"""  

    if self.session.query(exists().where(SKC.id==collection_id)).scalar():

      # remove collection - expressions relations
      q2 = self.session.query(SKC_HM) \
          .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
          .filter(SKC.id==collection_id)
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()

      # remove expression -> collection relations
      q3 = self.session.query(SKE_MO) \
          .filter(SKE_MO.member_of_id==collection_id)
      for cp in q3.all():
        self.session.delete(cp)
        self.session.flush()

      q4 = self.session.query(SKC) \
          .filter(SKC.id==collection_id)
      for c in q4.all():
        self.session.delete(c)
        self.session.flush()

      q5 = self.session.query(SKC_HN, N) \
        .filter(SKC_HN.ScientificKnowledgeCollection_id==collection_id) \
        .filter(SKC_HN.has_notes_id==N.id)
      for hn, n in q5.all():
        self.session.delete(hn)
        self.session.delete(n)
        self.session.flush()

      q6 = self.session.query(NoteIsAbout) \
        .filter(NoteIsAbout.Note_id.like('skc:'+collection_id+'.'))
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Collection does not exist')

  def delete_expression(self, expression_id, commit_this=True):    
    """Deletes the specified collection from the database, leave all expressions and items intact"""  

    if self.session.query(exists().where(SKE.id==expression_id)).scalar():

      q = self.session.query(SKI) \
          .filter(SKE_HR.ScientificKnowledgeExpression_id==expression_id) \
          .filter(SKE_HR.has_representation_id==SKI.id)
      for item in q.all():
        self.delete_item(item.id, commit_this=False)
        self.session.flush()

      # remove collection - expressions relations
      q2 = self.session.query(SKC_HM) \
          .filter(SKC_HM.has_members_id==expression_id)
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()

      # remove expression -> collection relations
      q3 = self.session.query(SKE_MO) \
          .filter(SKE_MO.ScientificKnowledgeExpression_id==expression_id)
      for cp in q3.all():
        self.session.delete(cp)
        self.session.flush()

      q5 = self.session.query(SKE_HN, N) \
        .filter(SKE_HN.ScientificKnowledgeExpression_id==expression_id) \
        .filter(SKC_HN.has_notes_id==N.id)
      for hn, n in q5.all():
        self.session.delete(hn)
        self.session.delete(n)
        self.session.flush()

      q6 = self.session.query(NoteIsAbout) \
        .filter(NoteIsAbout.Note_id.like('ske:'+expression_id+'.'))
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      q7 = self.session.query(SKE_IRI) \
        .filter(SKE_IRI.ScientificKnowledgeExpression_id==expression_id)
      for iri in q7.all():
        self.session.delete(iri)
        self.session.flush()

      q8 = self.session.query(SKE_XREF) \
        .filter(SKE_XREF.ScientificKnowledgeExpression_id==expression_id)
      for xref in q8.all():
        self.session.delete(xref)
        self.session.flush()

      q4 = self.session.query(SKE) \
          .filter(SKE.id==expression_id)
      for c in q4.all():
        self.session.delete(c)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Collection does not exist')

  def delete_item(self, item_id, commit_this=True):    
    """Deletes the specified item from the database, cascade the deletion to all attached fragments and notes."""  

    if self.session.query(exists().where(SKI.id==item_id)).scalar():

      q = self.session.query(SKF) \
          .filter(SKI_HP.ScientificKnowledgeItem_id==item_id) \
          .filter(SKI_HP.has_part_id==SKF.id)
      for fragment in q.all():
        self.delete_fragment(fragment.id, commit_this=False)
        self.session.flush()

      # remove expression -> item relations
      q2 = self.session.query(SKE_HR) \
          .filter(SKE_HR.has_representation_id==item_id)
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()

      q4 = self.session.query(SKI) \
          .filter(SKI.id==item_id)
      for c in q4.all():
        self.session.delete(c)
        self.session.flush()

      q5 = self.session.query(SKI_HN, N) \
        .filter(SKI_HN.ScientificKnowledgeItem_id==item_id) \
        .filter(SKI_HN.has_notes_id==N.id)
      for hn, n in q5.all():
        self.session.delete(hn)
        self.session.delete(n)
        self.session.flush()

      q6 = self.session.query(NoteIsAbout) \
        .filter(NoteIsAbout.Note_id.like('ski:'+item_id+'.'))
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Collection does not exist')

  def delete_fragment(self, fragment_id, commit_this=True):    
    """Deletes the specified fragment from the database, cascade the deletion to all associated notes."""  

    if self.session.query(exists().where(SKF.id==fragment_id)).scalar():

      # remove item -> fragment relations
      q2 = self.session.query(SKI_HP) \
          .filter(SKI_HP.has_part_id==fragment_id)
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()

      q4 = self.session.query(SKF) \
          .filter(SKF.id==fragment_id)
      for c in q4.all():
        self.session.delete(c)
        self.session.flush()

      q5 = self.session.query(SKF_HN, N) \
        .filter(SKF_HN.ScientificKnowledgeFragment_id==fragment_id) \
        .filter(SKF_HN.has_notes_id==N.id)
      for hn, n in q5.all():
        self.session.delete(hn)
        self.session.delete(n)
        self.session.flush()

      q6 = self.session.query(NoteIsAbout) \
        .filter(NoteIsAbout.Note_id.like('skf:'+fragment_id+'.'))
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Collection does not exist')  

  def delete_note(self, note_id, commit_this=True):    
    """Deletes the specified note from the database, cascade the deletion to all associated entities."""  

    if self.session.query(exists().where(Note.id==note_id)).scalar():

      # remove item -> fragment relations
      q1 = self.session.query(NoteIsAbout) \
          .filter(NoteIsAbout.Note_id==note_id) 
      for cp in q1.all():
        self.session.delete(cp)
        self.session.flush()

      q2 = self.session.query(SKF_HN) \
          .filter(SKF_HN.has_notes_id==note_id) 
      for cp in q2.all():
        self.session.delete(cp)
        self.session.flush()
 
      q3 = self.session.query(SKC_HN) \
          .filter(SKC_HN.has_notes_id==note_id) 
      for cp in q3.all():
        self.session.delete(cp)
        self.session.flush()

      q4 = self.session.query(SKE_HN) \
          .filter(SKE_HN.has_notes_id==note_id) 
      for cp in q4.all():
        self.session.delete(cp)
        self.session.flush()

      q5 = self.session.query(SKI_HN) \
          .filter(SKI_HN.has_notes_id==note_id) 
      for cp in q5.all():
        self.session.delete(cp)
        self.session.flush()

      q6 = self.session.query(N_HN) \
          .filter(N_HN.has_notes_id==note_id) 
      for cp in q6.all():
        self.session.delete(cp)
        self.session.flush()

      q6 = self.session.query(Note) \
        .filter(Note.id==note_id)
      for nia in q6.all():
        self.session.delete(nia)
        self.session.flush()

      if commit_this:
        self.session.commit()
    
    else:
      raise Exception('Note does not exist')  

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Analyze the structure / composition of a collection
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def build_full_text_report_on_collection(self, collection_id):
    q = self.session.query(SKE) \
        .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
        .filter(SKC_HM.has_members_id==SKE.id) \
        .filter(SKC.id==str(collection_id)) \
        .order_by(desc(SKE.publication_date))
    return pd.DataFrame([self.check_full_text(e) for e in tqdm(q.all())])    

  def build_item_type_pivot_table(self):
    q = self.session.query(SKC.name, SKE.id, SKI.type) \
        .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
        .filter(SKC_HM.has_members_id==SKE.id) \
        .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
        .filter(SKE_HR.has_representation_id==SKI.id) 
    df = pd.DataFrame(q.all(), columns=['collection', 'doi', 'type'])    
    return df.pivot_table(index='collection', columns='type', values='doi', aggfunc=lambda x: len(x.unique()))

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Add full text elements to database based on dowloaded files
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def add_full_text_for_expression(self, e):
    """Adds Full Text Item data to given Expression 
    if full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = re.sub('doi:','',e.id)
    
    # Check local directory for files
    base_file_path = '%s%s/ft/'%(self.loc, self.name)
    nxml_file_path = base_file_path+doi+'.nxml'
    pdf_file_path = base_file_path+doi+'.pdf'
    html_file_path = base_file_path+doi+'.html'

    self.add_full_text_for_expression_nxml(e, nxml_file_path)
    self.add_full_text_for_expression_pdf(e, pdf_file_path)
    self.add_full_text_for_expression_html(e, html_file_path)
    self.session.commit() 

  def add_full_text_for_expression_nxml(self, e, nxml_file_path, update_existing=False):
    """Adds Full Text Item data to expression if *.nxml full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = re.sub('doi:','',e.id)

    q = self.session.query(SKI) \
        .filter(SKE_HR.ScientificKnowledgeExpression_id == e.id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(or_(SKI.type == 'JATSFullText'))
    if update_existing is False:
      for i in q.all():
        return
    else:
      for i in q.all():
        self.delete_item(i.id, commit_this=False)
        self.session.commit()

    if os.path.exists(nxml_file_path):
      with open(nxml_file_path, 'r') as f:
        xml = f.read()
        try:
          d = NxmlDoc(doi, xml)
        except Exception as ex:
          return
        doc_text = d.text 
        if 'body' in [so.element.tag for so in d.standoffs] is False:
          return
        ski_type = 'JATSFullText'
                
        ski_id = str(uuid.uuid4().hex)[:10]
        ski = ScientificKnowledgeItem(id=ski_id, 
                                      content=doc_text,
                                      xref=['file:'+nxml_file_path],
                                      type=ski_type,)
        e.has_representation.append(ski)
        self.session.add(ski)
        self.session.flush()
        self.add_sections_as_fragments_from_nxmldoc(ski, d)
        self.session.commit()

  def add_sections_as_fragments_from_nxmldoc(self, item, d):
    """Adds fragments from an NxmlDoc object to a ScientificKnowledgeItem object."""

    # ['PMID', 'PARAGRAPH_ID', 'TAG', 'TAG_TREE', 'OFFSET', 'LENGTH', 'FIG_REF', 'PLAIN_TEXT'
    fragments = []
    df = d.build_enhanced_document_dataframe()
    # df = df[df.SECTION_TREE!='']
    # Can search for a substring in top section headers
    # df3 = df[df.TOP_SECTION.str.contains('method', case=False)]

    try: 
      df3 = df[df.TOP_SECTION!='']
      df4 = df3.groupby('SECTION_TREE').agg({'OFFSET': 'min', 'LENGTH': 'sum', 'PLAIN_TEXT':lambda x: '\n'.join(x)}).sort_values('OFFSET')
      df4 = df4.reset_index(drop=False)
    
      if df4 is None:
        return
    
      for i, tup in df4.iterrows():      
        #sentence_split_text = '\n'.join(self.sent_detector.tokenize(tup.PLAIN_TEXT))
        sentence_split_text = tup.PLAIN_TEXT
        fragment = ScientificKnowledgeFragment(id=item.id[:10]+'.'+str(i), 
                                              type='section',
                                              offset=tup.OFFSET,
                                              length=tup.LENGTH,
                                              name=tup.SECTION_TREE,
                                              content=sentence_split_text)
        self.session.add(fragment)
        self.session.flush()
        item.has_part.append(fragment)
        fragment.part_of = item.id
        fragments.append(fragment)
        self.session.flush() 
      self.session.commit()
    except Exception as e:
      self.session.rollback()    
      
  def add_full_text_for_expression_pdf(self, e, pdf_file_path, update_existing=False):
    """Adds Full Text Item data to expression if *.pdf full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = re.sub('doi:','',e.id)

    q = self.session.query(SKI) \
        .filter(SKE_HR.ScientificKnowledgeExpression_id == e.id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(or_(SKI.type == 'PDFFullText'))
    if update_existing is False:
      for i in q.all():
        return
    else:
      for i in q.all():
        self.delete_item(i.id, commit_this=False)
        self.session.commit()

    if os.path.exists(pdf_file_path):
      if os.environ.get('HURIDOCS_NETWORK'):
        loader = HuridocsPDFLoader(pdf_file_path, host=os.environ.get('HURIDOCS_NETWORK'))
      else:   
        loader = HuridocsPDFLoader(pdf_file_path)
      #loader = LAPDFBlockLoader(pdf_file_path)
      docs = loader.load(curi_id='doi:'+doi)
      skfs = []
      doc_text = ''
      ski_type = 'PDFFullText'

      ski_id = str(uuid.uuid4().hex)[:10]
      for i, d in enumerate(docs):
        #sentence_split_text = '\n'.join(self.sent_detector.tokenize(d.page_content))
        sentence_split_text = d.page_content

        skf = ScientificKnowledgeFragment(id=ski_id+'.'+str(i), 
                                             type='section',
                                             offset=len(doc_text),
                                             length=len(sentence_split_text),
                                             name=sentence_split_text.split('\n')[0],
                                             content=sentence_split_text)
        if len(doc_text) > 0:
          doc_text += '\n\n'
        doc_text += sentence_split_text
        self.session.add(skf)
        self.session.flush()
        skfs.append(skf)
      ski = ScientificKnowledgeItem(id=ski_id, 
                                    content=doc_text,
                                    xref=['file:'+pdf_file_path],
                                    type=ski_type,)
      e.has_representation.append(ski)
      self.session.add(ski)
      self.session.flush()
      for skf in skfs:
        ski.has_part.append(skf)
        skf.part_of = ski.id
        self.session.flush()
  
  def add_full_text_for_expression_html(self, e, html_file_path, update_existing=False):
    """Adds Full Text Item data to expression if *.html full text is already downloaded in the `{loc}/ft/{doi}` directory."""

    doi = re.sub('doi:','',e.id)

    q = self.session.query(SKI) \
        .filter(SKE_HR.ScientificKnowledgeExpression_id == e.id) \
        .filter(SKE_HR.has_representation_id == SKI.id) \
        .filter(or_(SKI.type == 'HTMLFullText'))
    if update_existing is False:
      for i in q.all():
        return
    else:
      for i in q.all():
        self.delete_item(i.id, commit_this=False)
        self.session.commit()

    if os.path.exists(html_file_path):
      loader = TrafilaturaSectionLoader(html_file_path)
      #loader = LAPDFBlockLoader(pdf_file_path)
      try: 
        docs = loader.load()
      except Exception as ex:
        print('Error loading HTML file: %s'%(html_file_path))
        return
      skfs = []
      doc_text = ''
      ski_type = 'HTMLFullText'

      ski_id = str(uuid.uuid4().hex)[:10]
      for i, d in enumerate(docs):
        #sentence_split_text = '\n'.join(self.sent_detector.tokenize(d.page_content))
        sentence_split_text = d.page_content

        skf = ScientificKnowledgeFragment(id=ski_id+'.'+str(i), 
                                             type='section',
                                             offset=len(doc_text),
                                             length=len(sentence_split_text),
                                             name=sentence_split_text.split('\n')[0],
                                             content=sentence_split_text)
        if len(doc_text) > 0:
          doc_text += '\n\n'
        doc_text += sentence_split_text
        self.session.add(skf)
        self.session.flush()
        skfs.append(skf)
      ski = ScientificKnowledgeItem(id=ski_id, 
                                    content=doc_text,
                                    xref=['file:'+html_file_path],
                                    type=ski_type,)
      e.has_representation.append(ski)
      self.session.add(ski)
      self.session.flush()
      for skf in skfs:
        ski.has_part.append(skf)
        skf.part_of = ski.id
        self.session.flush()
        
  def add_full_text_for_collection(self, collection_id, 
                                   get_nxml=True, get_pdf=True, get_html=True):
    for e in self.list_expressions(collection_id=collection_id):
      self.add_full_text_for_expression(e, get_nxml, get_pdf, get_html)  
  
  def check_full_text(self, e):
      """Checks if full text is available and readable for a given expression."""

      doi = re.sub('doi:','',e.id)
      file_path = self.loc + self.name + '/ft/'
      nxml_file_path = file_path+doi+'.nxml'
      pdf_file_path = file_path+doi+'.pdf'
      html_file_path = file_path+doi+'.html'
      
      nxml_item_present = False
      q = self.session.query(SKI) \
          .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
          .filter(SKE_HR.has_representation_id==SKI.id) \
          .filter(SKI.type == 'JATSFullText') \
          .filter(SKE.id==e.id)
      if q.count() > 0:
          nxml_item_present = True
      nxml_present = os.path.exists(nxml_file_path)
      nxml_has_body = False
      if os.path.exists(nxml_file_path):
          with open(nxml_file_path, 'r') as file:
              soup = BeautifulSoup(file, "lxml-xml")
              if soup.body is not None:
                  nxml_has_body = True
      
      pdf_item_present = False
      q2 = self.session.query(SKI) \
          .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
          .filter(SKE_HR.has_representation_id==SKI.id) \
          .filter(SKI.type == 'PDFFullText') \
          .filter(SKE.id==e.id)
      if q2.count() > 0:
          pdf_item_present = True
      pdf_present = os.path.exists(pdf_file_path)
      pdf_readable = False
      if pdf_present:
          try: 
              with fitz.open(pdf_file_path) as doc:
                  pdf_readable = True
                  #print('PDF: %s, Page Count: %d'%(doi, doc.page_count))
          except Exception as ex:
              pdf_readable = False
                      
      if os.path.exists(html_file_path):
          html_present = True
      else:
          html_present = False

      html_readable = False
      if html_present:
        loader = TrafilaturaSectionLoader(html_file_path)
        try: 
          docs = loader.load()
          html_readable = True
        except Exception as ex:
          html_readable = False

      prior_attempts = False
      for n in self.read_notes_about_x(e):
        if 'download' in n.name:
          prior_attempts = True
          break

      report = {'doi': doi, 
                'nxml_item_present': nxml_item_present,
                'nxml_present': nxml_present,
                'nxml_has_body': nxml_has_body,
                'pdf_item_present': pdf_item_present,
                'pdf_present': pdf_present,
                'pdf_readable': pdf_readable,
                'html_present': html_present,
                'html_readable': html_readable,
                'prior_attempts': prior_attempts}
      
      return report

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Read and Write Notes for Entities
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def write_note_about_x(self, x, note_name, note_content, note_format, note_type, commit_this=False):
    """Writes a note about an entity in the database."""
    note_id = str(uuid.uuid4().hex)[:10]
    creation_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S") 
    note = Note(id=note_id, 
             name=note_name, 
             creation_date=creation_date,
             format=note_format,
             type=note_type, 
             content=note_content)
    self.session.add(note)
    x.has_notes.append(note)
    #note.is_about.append(x)
    self.session.flush()
    if commit_this:
      self.session.commit()
    return note
  
  def read_notes_about_x(self, x):
    """Reads notes about an entity in the database."""
    q = self.session.query(N) \
        .filter(N.id == NIA.Note_id) \
        .filter(NIA.is_about_id == x.id)
    for n in q.all():
      yield(n)

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Quick reports
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def report_collection_composition(self):
    """Returns a DataFrame with the composition of all collections in the database."""
    q = self.session.query(SKC.id, SKC.name, SKE.id, SKI.type) \
            .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
            .filter(SKC_HM.has_members_id==SKE.id) \
            .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id==SKI.id) 
    df = pd.DataFrame(q.all(), columns=['id', 'collection name', 'doi', 'item type'])
    piv_df = df.pivot_table(index=['id', 'collection name'], columns='item type', values='doi', aggfunc=lambda x: len(x.unique())).fillna(0)
    return piv_df

  def report_non_full_text_for_collection(self, collection_id):
    """Returns a DataFrame listing all papers without PDF or JATS full text versions in the database."""
    q = self.session.query(SKC.id, SKE.id, SKI.type) \
            .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
            .filter(SKC_HM.has_members_id==SKE.id) \
            .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id==SKI.id) \
            .filter(SKC.id==str(collection_id))
    df = pd.DataFrame(q.all(), columns=['collection_id', 'doi', 'item type'])
    piv_df = df.pivot_table(index=['doi'], 
                        columns='item type', values='collection_id', 
                        aggfunc=lambda x: len(x.unique()))
    piv_df.fillna(0, inplace=True)
    piv_df = piv_df.reset_index(drop=False).sort_values(by='doi')
    return piv_df[(piv_df['JATSFullText']==0.0) & (piv_df['PDFFullText']==0.0)]

  def report_all_notes_by_type_name(self, note_type, name_pattern):
    """Returns a DataFrame listing all note contents with full text versions in the database."""
    q = self.session.query(N) \
      .filter(N.id == NIA.Note_id) \
      .filter(N.type == note_type) \
      .filter(N.name.like(name_pattern)) 
    l = []
    for n in q.all():
        tup = json.loads(n.content)
        tup = json.loads(n.provenance)
        tup['note_id'] = n.name
        l.append(tup)
    report_df = pd.DataFrame(l).set_index('note_id')
    return report_df

  def create_zip_archive_of_full_text_files(self, collection_id, zip_file_path):
    """Creates a zip archive of all full text files for a given collection."""
    file_path = self.loc + self.name + '/ft/'
    with ZipFile(zip_file_path, 'w') as myzip:
      for e in self.list_expressions(collection_id=collection_id):
        doi = re.sub('doi:','',e.id)
        nxml_file_path = file_path+doi+'.nxml'
        pdf_file_path = file_path+doi+'.pdf'
        html_file_path = file_path+doi+'.html'
        if os.path.exists(nxml_file_path):
          myzip.write(nxml_file_path, arcname=doi+'.nxml')
        if os.path.exists(pdf_file_path):
          myzip.write(pdf_file_path, arcname=doi+'.pdf')
        if os.path.exists(html_file_path):
          myzip.write(html_file_path, arcname=doi+'.html')
          
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Query Collections, Expressions, Items, and Fragments
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def get_expression_by_doi(self, doi):
    """Returns a ScientificKnowledgeExpression object for a given DOI."""
    self.start_session()
    query = self.session.query(SKE) \
        .filter(SKE.id==SKE_XREF.ScientificKnowledgeExpression_id) \
        .filter(SKE_XREF.xref=='doi:'+doi)
    ske = query.first()
    return ske

  def list_items_for_expression(self, doi):
    """Returns all items for an expression for a given DOI."""
    self.start_session()
    if doi[:4] != 'doi:':
      doi = 'doi:'+doi
    query = self.session.query(SKE, SKI) \
          .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
          .filter(SKE_HR.has_representation_id == SKI.id) \
          .filter(SKE.id == doi)
    l_it = []
    for (ex, it) in query.all():
      l_it.append(it)
    return l_it

  def list_collections(self, search_term=None):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if search_term:
      q = (self.session.query(SKC) 
          .filter(SKC.name == search_term))
    else:
      q = self.session.query(SKC)
    for c in q.all():
      yield(c)

  def list_expressions(self, collection_id=None, search_term=None): 
    '''Lists expressions in database. 
    Including the `collection_id` parameter limits the returned list to the specified collection. 
    Including the `search_term` parameter searches the citation string for the specified term.'''

    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if collection_id:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(SKE) 
            .filter(SKC_HM.ScientificKnowledgeCollection_id==collection_id)
            .filter(SKC_HM.has_members_id == SKE.id)
            .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id)
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.content.like(search))
          )
      else:
         q = (self.session.query(SKE) 
            .filter(SKC.id == SKC_HM.ScientificKnowledgeCollection_id)
            .filter(SKC_HM.has_members_id == SKE.id)
            .filter(SKC.id == collection_id)
          )
    else:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(SKE) 
            .filter(SKC_HM.has_members_id == SKE.id)
            .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id)
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.content.like(search))
          )
      else:
        q = self.session.query(SKE)
      
    for c in q.all():
      yield(c)

  def list_notes_for_fragments_in_paper(self, run_name, paper_id, item_type='FullTextPaper'):
    '''returns notes of a specific type associated with fragments from a given paper .'''
    q1 = self.session.query(SKI) \
            .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id == SKI.id) \
            .filter(SKI.type == item_type) \
            .filter(SKE.id.like('%'+str(paper_id)+'%')) 
    i = q1.first()
    if i is None:
      return []
    for f in i.has_part:
      for n in f.has_notes:
        if n.name == run_name:
          yield(n)
         
  def list_fragments_for_paper(self, paper_id, item_type, fragment_types=['title', 'abstract']):
    '''Loads fragments from a given paper sections of a specified paper from the local database.'''
    q1 = self.session.query(SKI) \
            .filter(SKE.id == SKE_HR.ScientificKnowledgeExpression_id) \
            .filter(SKE_HR.has_representation_id == SKI.id) \
            .filter(SKI.type == item_type) \
            .filter(SKE.id.like('%'+str(paper_id)+'%')) 
    i = q1.first()
    if i is None:
      return []
    fragments = []
    for f in i.has_part:
      if f.type in fragment_types:
        fragments.append(f)
    return sorted(fragments, key=lambda f: f.offset)
  
  def list_fragments(self, expression_id=None, search_term=None):
    if self.session is None:
      session_class = sessionmaker(bind=self.engine)
      self.session = session_class()
    if expression_id:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(SKF) 
            .filter(SKE_HR.ScientificKnowledgeExpression_id == expression_id)
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id)
            .filter(SKI_HP.has_part_id == SKF.id)
            .filter(SKF.content.ilike('%'+search+'%'))
          )
      else:
        q = (self.session.query(SKF) 
            .filter(SKE_HR.ScientificKnowledgeExpression_id == expression_id)
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id)
            .filter(SKI_HP.has_part_id == SKF.id)
          )
    else:
      if search_term:
        search = "%{}%".format(search_term)
        q = (self.session.query(SKF) 
            .filter(SKE_HR.has_representation_id == SKI.id)
            .filter(SKI.id == SKI_HP.ScientificKnowledgeItem_id)
            .filter(SKI_HP.has_part_id == SKF.id)
            .filter(SKF.content.ilike('%'+search+'%'))
          )
      else:
        q = self.session.query(SKF)
    for c in q.all():
      yield(c)

  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Build RAG Indexes of documents
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
  def embed_expression_list(self, explist):
    '''Runs through the list of expressions, generates embeddings, and stores them in the database'''

    texts = []
    metadatas = []
    
    for e in tqdm(explist):
      coll_ids = ','.join([c.id for c in e.member_of])

      t, a = self.list_fragments_for_paper(e.id, 'CitationRecord')
      i = t.part_of
      tiab_text = t.content+'\n\n'+a.content

      if len(tiab_text) > 0:
        texts.append(tiab_text)
        metadatas.append({'c_ids': coll_ids, \
                                  'e_id': e.id, \
                                  'e_type': e.type, \
                                  'i_id': t.part_of, \
                                  'i_type': 'CitationRecord', \
                                  'f_id': t.id, \
                                  'f_type': t.type+','+a.type, \
                                  'citation': e.content})

      jats_frgs = self.list_fragments_for_paper(e.id, 'JATSFullText', ['section'])
      for f in jats_frgs:
        texts.append(f.content)
        metadatas.append({'c_ids': coll_ids, \
                               'e_id': e.id, \
                               'e_type': e.type, \
                               'i_id': f.part_of, \
                               'i_type': 'JATSFullText', \
                               'f_id': f.id, \
                               'citation': e.content})        

      pdf_frgs = self.list_fragments_for_paper(e.id, 'PDFFullText', ['section'])
      for f in pdf_frgs:
        texts.append(f.content)
        metadatas.append({'c_ids': coll_ids, \
                              'e_id': e.id, \
                              'e_type': e.type, \
                              'i_id': f.part_of, \
                              'i_type': 'PDFFullText', \
                              'f_id': f.id, \
                              'citation': e.content})

    # Use this when we want to index text on individual sentences       
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size = 3000, chunk_overlap=150)
    
    # build vector indexes from citation record     
    #tiab_docs = text_splitter.create_documents(tiab_texts, metadatas=tiab_metadatas)
    docs = []
    for t,m in zip(texts, metadatas):
      docs.append(Document(page_content=t, metadata=m))
    
    print('Indexing %d documents'%(len(docs)))
    db = PGVector.from_documents(
        embedding=self.embed_model,
        documents=docs,
        collection_name="ScienceKnowledgeItem"
    )

  def query_vectorindex(self, query_string, entity_type='ScienceKnowledgeItem', k=4):
    db2 = PGVector.from_existing_index(
      embedding=self.embed_model, 
      collection_name=entity_type) 
    docs = db2.similarity_search_with_score(query_string, k=k)
    return docs
  
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # Retrieve and manipulate raw vectors
  # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  def list_unindexed_skes(self, collection_id):
    query = """
        SELECT DISTINCT ske.id
        FROM "ScientificKnowledgeExpression" as ske
            JOIN "ScientificKnowledgeCollection_has_members" as skc_hm on ske.id = skc_hm.has_members_id
            LEFT JOIN langchain_pg_embedding as emb on emb.cmetadata->>'e_id' = ske.id
        WHERE emb.cmetadata->>'e_id' IS NULL AND
            skc_hm."ScientificKnowledgeCollection_id"='{}';
        """.format(collection_id)
    skes = []
    for ske_id in self.session.execute(text(query)).fetchall():
      skes.extend(self.session.query(SKE).filter(SKE.id==ske_id[0]).all())
    return skes
  
  def list_embedded_skes_for_collection_by_fragment_type(self, collection_id, f_type='title,abstract'):
    query = """
        SELECT DISTINCT ske.id, emb.uid, emb.cmetadata->>'f_id'
        FROM langchain_pg_embedding as emb, 
            "ScientificKnowledgeExpression" as ske,
            "ScientificKnowledgeCollection_has_members" as skc_hm
        WHERE emb.cmetadata->>'f_type' = '{}}' AND
            emb.cmetadata->>'e_id' = ske.id AND 
            ske.id = skc_hm.has_members_id AND
            skc_hm."ScientificKnowledgeCollection_id"='{}';
        """.format(f_type, collection_id)
    rag_data = self.session.execute(text(query)).fetchall()
    return rag_data

  def delete_repeated_embeddings_for_collection_by_fragment_type(self, collection_id, f_type='title,abstract'):
    rag_data = self.list_embedded_skes_for_collection_by_fragment_type(collection_id, f_type=f_type)
    checked_dois = []
    to_remove = []
    for tup in rag_data:
        if tup[2] not in checked_dois:
            checked_dois.append(tup[2])
        else:
            to_remove.append(tup[1])
    for Uuid in to_remove:
      self.session.execute(text('DELETE FROM langchain_pg_embedding WHERE uuid=:uuid'), {'uuid':Uuid})
    self.session.commit()

  def retrieve_vectors_for_collection_by_item_type(self, collection_id, i_type='CitationRecord'):
    query = """
        SELECT DISTINCT ske.id, emb.embedding 
        FROM langchain_pg_embedding as emb, 
            "ScientificKnowledgeExpression" as ske,
            "ScientificKnowledgeCollection_has_members" as skc_hm
        WHERE emb.cmetadata->>'i_type' = '{}}' AND
            emb.cmetadata->>'e_id' = ske.id AND 
            ske.id = skc_hm.has_members_id AND
            skc_hm."ScientificKnowledgeCollection_id"='{}';
        """.format(i_type, collection_id)
    rag_data = self.session.execute(text(query)).fetchall()
    return rag_data

  def retrieve_vectors_for_collection_by_fragment_type(self, collection_id, f_type='title,abstract'):
    query = """
        SELECT DISTINCT ske.id, ske.content, emb.embedding 
        FROM langchain_pg_embedding as emb, 
            "ScientificKnowledgeExpression" as ske,
            "ScientificKnowledgeCollection_has_members" as skc_hm
        WHERE emb.cmetadata->>'f_type' = '{}}' AND
            emb.cmetadata->>'e_id' = ske.id AND 
            ske.id = skc_hm.has_members_id AND
            skc_hm."ScientificKnowledgeCollection_id"='{}';
        """.format(f_type, collection_id)
    rag_data = self.session.execute(text(query)).fetchall()
    return rag_data

`>>>>>>> 75dc97aff9dc932799675920edfaed089bf106a3`

In [ ]:
#| export

def read_information_content_entity_iri(ice, id_prefix):
    """Reads an identifier for a given prefix"""
    idmap = {k[:k.find(':')]:k[k.find(':')+1:] for k in ice.xref} 
    return idmap.get(id_prefix)